# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [25]:
import pandas as pd
import numpy as np
import xlsxwriter
import requests 
from scipy import stats 
import math 


## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [26]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN


## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [27]:
#'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
symbol = 'AAPL'

api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)
#latestUpdate, peRatio


{'avgTotalVolume': 66954879, 'calculationPrice': 'close', 'change': -2.15, 'changePercent': -0.01361, 'close': 155.81, 'closeSource': 'official', 'closeTime': 1662148800416, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 155.615, 'delayedPriceTime': 1662148781927, 'extendedChange': -0.16, 'extendedChangePercent': -0.00103, 'extendedPrice': 155.65, 'extendedPriceTime': 1662163197667, 'high': 160.362, 'highSource': '15 minute delayed price', 'highTime': 1662148799986, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 155.845, 'iexCloseTime': 1662148799399, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 159.87, 'iexOpenTime': 1662125400491, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1662148799986, 'latestPrice': 155.81, 'latestSource': 'Close', 'latestTime': 'September 2, 2022', 'latestUpdate': 1662148800416, 'latestVolume': 76957768, 'low': 154.965, 'lowSource': '15 minut

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [28]:
price = data['latestPrice']
pe_ratio = data['peRatio']
print(pe_ratio)

25.75


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [29]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [30]:
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe 

for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try: 
            final_dataframe = final_dataframe.append(
                pd.Series(
                    [ 
                        symbol,
                        data[symbol]['quote']['latestPrice'],
                        data[symbol]['quote']['peRatio'],
                        'N/A'
                    ], index = my_columns,
                ), ignore_index = True 
            )
        except:
            print(symbol)
            pass
final_dataframe

C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Tem

VIAC
WLTW


C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\2155335645.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
C:\Users\Sunny\AppData\Local\Tem

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,128.01,29.16,N/A
1,AAL,12.99,-4.48,N/A
2,AAP,171.73,19.97,N/A
3,AAPL,155.81,25.75,N/A
4,ABBV,136.28,19.28,N/A
...,...,...,...,...
496,YUM,112.74,22.5,N/A
497,ZBH,105.57,97.75,N/A
498,ZBRA,297.6,31.66,N/A
499,ZION,55.04,6.18,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [31]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True) #can be done in above step. 
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.47,2.92,N/A
1,AIG,52.41,3.45,N/A
2,PVH,54.58,3.93,N/A
3,NUE,130.78,4.03,N/A
4,APA,38.8,4.51,N/A
5,PHM,40.45,4.51,N/A
6,COF,104.22,4.55,N/A
7,DHI,71.38,4.59,N/A
8,PFG,74.44,4.63,N/A
9,SYF,31.97,4.65,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [32]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")



Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [33]:
portfolio_size = 1000000

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [34]:
position_size = portfolio_size/len(final_dataframe.index)
position_size 

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe 

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,41.47,2.92,482
1,AIG,52.41,3.45,381
2,PVH,54.58,3.93,366
3,NUE,130.78,4.03,152
4,APA,38.8,4.51,515
5,PHM,40.45,4.51,494
6,COF,104.22,4.55,191
7,DHI,71.38,4.59,280
8,PFG,74.44,4.63,268
9,SYF,31.97,4.65,625


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [35]:
symbol = 'AAPL'
import random #random will be used instead of premium API calls. 

#since we're not using the premium version, we have to exclude the advanced-stats endpoint. Commented out the batch_url_call with advanced-stats endpoint. 
#batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_url).json()

# * Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# * Price-to-book ratio
#pb_ratio = data[symbol]['advanced-stats']['priceToBook']   #advanced-stats endpoint = paid subscription. 
pb_ratio = np.NaN 

# * Price-to-sales ratio
#ps_ratio = data[symbol]['advanced-stats']['priceToSales']  #advanced-stats endpoint = paid subscription. 
ps_ratio = np.NaN

# * Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
#enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']    #advanced-stats endpoint = paid subscription. 
#ebitda = data[symbol]['advanced-stats']['EBITDA']                       #advanced-stats endpoint = paid subscription. 
#ev_to_ebitda = enterprise_value/ebitda                                  #advanced-stats endpoint = paid subscription. 
ev_to_ebitda = np.NaN

# * Enterprise Value divided by Gross Profit (EV/GP)
#gross_profit = data[symbol]['advanced-stats']['grossProfit']            #advanced-stats or financial endpoint both = paid subscription.       
#ev_to_gross_profit = enterprise_value/gross_profit                      #advanced-stats or financial endpoint both = paid subscription.       
ev_to_gross_profit = np.NaN


Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [36]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        try:
            rv_dataframe = rv_dataframe.append (
                pd.Series (
                    [ 
                            symbol,
                            data[symbol]['quote']['latestPrice'],
                            'N/A',
                            data[symbol]['quote']['peRatio'],
                            'PE Percentile',
                            random.randrange(8,30),     #premium api call required, using random instead.
                            'PB Percentile',
                            random.randrange(8,30),     #premium api call required, using random instead.
                            'PS Percentile',
                            random.randrange(8,30),     #premium api call required, using random instead.
                            'EV/EBITDA Percentile',
                            random.randrange(8,30),     #premium api call required, using random instead.
                            'EV/GP Percentile',
                            random.randrange(8,30),     #premium api call required, using random instead.
                    ], index = rv_columns
                ), ignore_index = True 
            )
        except:
            pass 


C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\3213559593.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append (
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\3213559593.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append (
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\3213559593.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append (
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\3213559593.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rv_dataframe = rv_dataframe.append (
C:\Users\Sunny\AppData\Local\Temp\ipykernel_616\

In [37]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.01,N/A,29.16,PE Percentile,28,PB Percentile,9,PS Percentile,9,EV/EBITDA Percentile,16,EV/GP Percentile,9
1,AAL,12.99,N/A,-4.48,PE Percentile,22,PB Percentile,11,PS Percentile,22,EV/EBITDA Percentile,16,EV/GP Percentile,20
2,AAP,171.73,N/A,19.97,PE Percentile,12,PB Percentile,20,PS Percentile,14,EV/EBITDA Percentile,27,EV/GP Percentile,28
3,AAPL,155.81,N/A,25.75,PE Percentile,23,PB Percentile,11,PS Percentile,28,EV/EBITDA Percentile,14,EV/GP Percentile,17
4,ABBV,136.28,N/A,19.28,PE Percentile,25,PB Percentile,13,PS Percentile,17,EV/EBITDA Percentile,16,EV/GP Percentile,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.74,N/A,22.5,PE Percentile,22,PB Percentile,21,PS Percentile,27,EV/EBITDA Percentile,12,EV/GP Percentile,22
497,ZBH,105.57,N/A,97.75,PE Percentile,19,PB Percentile,29,PS Percentile,27,EV/EBITDA Percentile,20,EV/GP Percentile,19
498,ZBRA,297.6,N/A,31.66,PE Percentile,24,PB Percentile,12,PS Percentile,29,EV/EBITDA Percentile,8,EV/GP Percentile,23
499,ZION,55.04,N/A,6.18,PE Percentile,18,PB Percentile,10,PS Percentile,16,EV/EBITDA Percentile,15,EV/GP Percentile,12


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [55]:
#return rows with missing data 
rv_dataframe[rv_dataframe.isnull().any(axis=1)]
#return # of rows with missing data
len(rv_dataframe[rv_dataframe.isnull().any(axis=1)].index)

10

Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [57]:
#remember inplace = True
for column in ['Price-to-Earnings Ratio','Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP', 'RV Score']: 
    #print(column)
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [58]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [64]:
metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio': 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])


rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.01,N/A,29.16,72.255489,28,93.313373,9,6.087824,9,5.489022,16,42.215569,9
1,AAL,12.99,N/A,-4.48,3.393214,22,68.962076,11,14.870259,22,64.97006,16,42.215569,20
2,AAP,171.73,N/A,19.97,49.9002,12,21.556886,20,57.285429,14,28.642715,27,90.219561,28
3,AAPL,155.81,N/A,25.75,65.668663,23,73.253493,11,14.870259,28,92.914172,14,31.437126,17
4,ABBV,136.28,N/A,19.28,47.904192,25,81.936128,13,26.247505,17,41.716567,16,42.215569,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.74,N/A,22.50,54.690619,22,68.962076,21,62.075848,27,88.622754,12,21.856287,22
497,ZBH,105.57,N/A,97.75,97.804391,19,55.788423,29,98.403194,27,88.622754,20,58.882236,19
498,ZBRA,297.6,N/A,31.66,75.648703,24,77.54491,12,20.259481,29,97.804391,8,2.095808,23
499,ZION,55.04,N/A,6.18,9.181637,18,49.500998,10,10.07984,16,37.824351,15,36.726547,12


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [69]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentile = []
    for metric in metrics.keys():
        value_percentile.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentile)
rv_dataframe 

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,128.01,N/A,29.16,72.255489,28,93.313373,9,6.087824,9,5.489022,16,42.215569,43.872255
1,AAL,12.99,N/A,-4.48,3.393214,22,68.962076,11,14.870259,22,64.97006,16,42.215569,38.882236
2,AAP,171.73,N/A,19.97,49.9002,12,21.556886,20,57.285429,14,28.642715,27,90.219561,49.520958
3,AAPL,155.81,N/A,25.75,65.668663,23,73.253493,11,14.870259,28,92.914172,14,31.437126,55.628743
4,ABBV,136.28,N/A,19.28,47.904192,25,81.936128,13,26.247505,17,41.716567,16,42.215569,48.003992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,112.74,N/A,22.50,54.690619,22,68.962076,21,62.075848,27,88.622754,12,21.856287,59.241517
497,ZBH,105.57,N/A,97.75,97.804391,19,55.788423,29,98.403194,27,88.622754,20,58.882236,79.900200
498,ZBRA,297.6,N/A,31.66,75.648703,24,77.54491,12,20.259481,29,97.804391,8,2.095808,54.670659
499,ZION,55.04,N/A,6.18,9.181637,18,49.500998,10,10.07984,16,37.824351,15,36.726547,28.662675


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [74]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
#keep top 50
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace = True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GLW,33.15,N/A,13.870000,30.938124,8,2.894212,11,14.870259,11,14.770459,8,2.095808,13.113772
1,AXP,149.79,N/A,15.360000,35.329341,8,2.894212,13,26.247505,8,1.796407,9,6.986028,14.650699
2,MHK,108.85,N/A,7.320000,12.57485,13,26.646707,8,2.195609,10,10.179641,15,36.726547,17.664671
3,KEY,17.55,N/A,7.660000,13.572854,8,2.894212,12,20.259481,10,10.179641,16,42.215569,17.824351
4,FCX,28.3,N/A,8.580000,14.570858,9,8.083832,9,6.087824,18,46.906188,13,26.546906,20.439122
5,MS,86.4,N/A,11.660000,24.151697,10,12.47505,12,20.259481,13,24.850299,12,21.856287,20.718563
6,SYF,31.97,N/A,4.650000,6.586826,18,49.500998,9,6.087824,13,24.850299,11,17.365269,20.878244
7,AES,26.15,N/A,-50.130000,1.197605,10,12.47505,13,26.247505,10,10.179641,19,54.491018,20.918164
8,WY,33.87,N/A,10.330000,19.361277,15,36.327345,8,2.195609,15,33.433134,13,26.546906,23.572854
9,AMAT,91.24,N/A,12.200000,25.548902,14,31.337325,14,30.738523,12,19.860279,10,12.47505,23.992016


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [75]:
portfolio_size = 100000

In [95]:
position_size = portfolio_size/len(rv_dataframe)

for row in range(len(rv_dataframe)):
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])/100

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,GLW,33.15,0.6,13.870000,30.938124,8,2.894212,11,14.870259,11,14.770459,8,2.095808,13.113772
1,AXP,149.79,0.13,15.360000,35.329341,8,2.894212,13,26.247505,8,1.796407,9,6.986028,14.650699
2,MHK,108.85,0.18,7.320000,12.57485,13,26.646707,8,2.195609,10,10.179641,15,36.726547,17.664671
3,KEY,17.55,1.13,7.660000,13.572854,8,2.894212,12,20.259481,10,10.179641,16,42.215569,17.824351
4,FCX,28.3,0.7,8.580000,14.570858,9,8.083832,9,6.087824,18,46.906188,13,26.546906,20.439122
5,MS,86.4,0.23,11.660000,24.151697,10,12.47505,12,20.259481,13,24.850299,12,21.856287,20.718563
6,SYF,31.97,0.62,4.650000,6.586826,18,49.500998,9,6.087824,13,24.850299,11,17.365269,20.878244
7,AES,26.15,0.76,-50.130000,1.197605,10,12.47505,13,26.247505,10,10.179641,19,54.491018,20.918164
8,WY,33.87,0.59,10.330000,19.361277,15,36.327345,8,2.195609,15,33.433134,13,26.546906,23.572854
9,AMAT,91.24,0.21,12.200000,25.548902,14,31.337325,14,30.738523,12,19.860279,10,12.47505,23.992016


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [98]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name = 'Value Strategy', index = False)


## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [99]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [100]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['Price-to-Earnings Ratio', float_template],
    'E': ['PE Percentile', percent_template],
    'F': ['Price-to-Book Ratio', float_template],
    'G': ['PB Percentile', percent_template],
    'H': ['Price-to-Sales Ratio', float_template],
    'I': ['PS Percentile', percent_template],
    'J': ['EV/EBITDA', float_template],
    'K': ['EV/EBITDA Percentile', percent_template],
    'L': ['EV/GP', float_template],
    'M': ['EV/GP Percentile', percent_template],
    'N': ['RV Score', float_template]
}


for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])



## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()